In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('crowdfunding_preprocess1.csv', index_col=0)
df.head()

,id,funded_amount,funding_goal,successful_funding,activity,sector,use,use_description,country_code,country,region,currency,partner_id,term_in_months,lender_count,borrower_genders,count_male,count_female,repayment_interval
0,0,300.0,300.0,goal_achieved,Fruits & Vegetables,Food,"To buy seasonal, fresh fruits to sell.",1,PK,Pakistan,Lahore,PKR,247.0,12.0,12,female,0,1,irregular
1,1,575.0,575.0,goal_achieved,Rickshaw,Transportation,to repair and maintain the auto rickshaw used ...,1,PK,Pakistan,Lahore,PKR,247.0,11.0,14,"female, female",0,2,irregular
2,2,150.0,150.0,goal_achieved,Transportation,Transportation,To repair their old cycle-van and buy another ...,1,IN,India,Maynaguri,INR,334.0,43.0,6,female,0,1,bullet
3,3,200.0,200.0,goal_achieved,Embroidery,Arts,to purchase an embroidery machine and a variet...,1,PK,Pakistan,Lahore,PKR,247.0,11.0,8,female,0,1,irregular
4,4,400.0,400.0,goal_achieved,Milk Sales,Food,to purchase one buffalo.,1,PK,Pakistan,Abdul Hakeem,PKR,245.0,14.0,16,female,0,1,monthly


# 3. Neue Spalten

- 3.1 successful_funding: sinnvoll, um auszuwerten, wann funding stattfindet und wann nicht
    - no_funding: wenn funded_amount = 0
    - goal_not_achieved: wenn größer 0 und kleiner funding_goal
    - goal_achieved: wenn funded_amount = funding_goal
    - goal_overachieved: wenn funded_amount > funding_goal
- 3.2 use_description: sinnvoll, um zu sehen, ob Projekte mit Beschreibung eher Funding erhalten als ohne Beschreibung
    - ja/nein (non-null / null)
- 3.3 count_initiators: wieviele Initiatoren das Projekt leiten (Anzahl der Einträge in der Spalte borrower_genders)
- 3.4 gender_initiators: weiblich/männlich/gemischt (evtl.)
- 3.5 funded_cluster: Da funded_amount so viele Werte hat, ist die Spalte so für die Visualisierung nicht brauchbar, - 3.5 - 3.6 goal_cluster: Da funded_amount so viele Werte hat, ist die Spalte so für die Visualisierung nicht brauchbar, deshalb werden die Werte mit der cut-Funktion gruppiert in mehrere Bins
- 3.7 successful_percent: Anteil des funded_amount an funding_goal in Prozent. Für spätere Visualisierung.

## 3.3 count_initiators

In [3]:
df.insert(loc=18 , column='count_initiators', value=0)

In [4]:
df.count_initiators = df.count_male + df.count_female

## 3.4 gender_initiators

In [5]:
df.insert(loc=19 , column='gender_initiators', value='')

In [6]:
df.loc[(df['count_male'] > 0) & (df['count_female']==0), 'gender_initiators'] = 'male'

In [7]:
df.loc[(df['count_female'] > 0) & (df['count_male']==0), 'gender_initiators'] = 'female'

In [8]:
df.loc[(df['count_male'] > 0) & (df['count_female']>0), 'gender_initiators'] = 'mixed'

In [9]:
#df.head()

## 3.5 funded_cluster

In [10]:
df.insert(loc=2 , column='funded_cluster', value=0)

In [11]:
df.funded_cluster= pd.cut(df['funded_amount'], bins=[-1,0,100,200,500,1000,2000,5000,10000,20000,50000,100000])
df

,id,funded_amount,funded_cluster,funding_goal,successful_funding,activity,sector,use,use_description,country_code,...,currency,partner_id,term_in_months,lender_count,borrower_genders,count_male,count_female,count_initiators,gender_initiators,repayment_interval
0,0,300.0,"(200, 500]",300.0,goal_achieved,Fruits & Vegetables,Food,"To buy seasonal, fresh fruits to sell.",1,PK,...,PKR,247.0,12.0,12,female,0,1,1,female,irregular
1,1,575.0,"(500, 1000]",575.0,goal_achieved,Rickshaw,Transportation,to repair and maintain the auto rickshaw used ...,1,PK,...,PKR,247.0,11.0,14,"female, female",0,2,2,female,irregular
2,2,150.0,"(100, 200]",150.0,goal_achieved,Transportation,Transportation,To repair their old cycle-van and buy another ...,1,IN,...,INR,334.0,43.0,6,female,0,1,1,female,bullet
3,3,200.0,"(100, 200]",200.0,goal_achieved,Embroidery,Arts,to purchase an embroidery machine and a variet...,1,PK,...,PKR,247.0,11.0,8,female,0,1,1,female,irregular
4,4,400.0,"(200, 500]",400.0,goal_achieved,Milk Sales,Food,to purchase one buffalo.,1,PK,...,PKR,245.0,14.0,16,female,0,1,1,female,monthly
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
671199,671199,0.0,"(-1, 0]",25.0,no_funding,Livestock,Agriculture,"[True, u'para compara: cemento, arenya y ladri...",1,PY,...,USD,58.0,13.0,0,female,0,1,1,female,monthly
671200,671200,0.0,"(-1, 0]",25.0,no_funding,Livestock,Agriculture,"[True, u'para compara: cemento, arenya y ladri...",1,PY,...,USD,58.0,13.0,0,female,0,1,1,female,monthly
671201,671201,25.0,"(0, 100]",25.0,goal_achieved,Livestock,Agriculture,"[True, u'to start a turducken farm.'] - this l...",1,KE,...,KES,138.0,13.0,1,female,0,1,1,female,monthly
671203,671203,0.0,"(-1, 0]",25.0,no_funding,Livestock,Agriculture,"[True, u'to start a turducken farm.'] - this l...",1,KE,...,KES,138.0,13.0,0,female,0,1,1,female,monthly


## 3.6 goal_cluster

In [12]:
df.insert(loc=4 , column='goal_cluster', value=0)

In [13]:
df.goal_cluster = pd.cut(df['funding_goal'], bins=[-1,0,100,200,500,1000,2000,5000,10000,20000,50000,100000])
df

,id,funded_amount,funded_cluster,funding_goal,goal_cluster,successful_funding,activity,sector,use,use_description,...,currency,partner_id,term_in_months,lender_count,borrower_genders,count_male,count_female,count_initiators,gender_initiators,repayment_interval
0,0,300.0,"(200, 500]",300.0,"(200, 500]",goal_achieved,Fruits & Vegetables,Food,"To buy seasonal, fresh fruits to sell.",1,...,PKR,247.0,12.0,12,female,0,1,1,female,irregular
1,1,575.0,"(500, 1000]",575.0,"(500, 1000]",goal_achieved,Rickshaw,Transportation,to repair and maintain the auto rickshaw used ...,1,...,PKR,247.0,11.0,14,"female, female",0,2,2,female,irregular
2,2,150.0,"(100, 200]",150.0,"(100, 200]",goal_achieved,Transportation,Transportation,To repair their old cycle-van and buy another ...,1,...,INR,334.0,43.0,6,female,0,1,1,female,bullet
3,3,200.0,"(100, 200]",200.0,"(100, 200]",goal_achieved,Embroidery,Arts,to purchase an embroidery machine and a variet...,1,...,PKR,247.0,11.0,8,female,0,1,1,female,irregular
4,4,400.0,"(200, 500]",400.0,"(200, 500]",goal_achieved,Milk Sales,Food,to purchase one buffalo.,1,...,PKR,245.0,14.0,16,female,0,1,1,female,monthly
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
671199,671199,0.0,"(-1, 0]",25.0,"(0, 100]",no_funding,Livestock,Agriculture,"[True, u'para compara: cemento, arenya y ladri...",1,...,USD,58.0,13.0,0,female,0,1,1,female,monthly
671200,671200,0.0,"(-1, 0]",25.0,"(0, 100]",no_funding,Livestock,Agriculture,"[True, u'para compara: cemento, arenya y ladri...",1,...,USD,58.0,13.0,0,female,0,1,1,female,monthly
671201,671201,25.0,"(0, 100]",25.0,"(0, 100]",goal_achieved,Livestock,Agriculture,"[True, u'to start a turducken farm.'] - this l...",1,...,KES,138.0,13.0,1,female,0,1,1,female,monthly
671203,671203,0.0,"(-1, 0]",25.0,"(0, 100]",no_funding,Livestock,Agriculture,"[True, u'to start a turducken farm.'] - this l...",1,...,KES,138.0,13.0,0,female,0,1,1,female,monthly


## 3.7 successful_percent

In [14]:
df.insert(loc=6 , column='success_factor', value=0)
df.head()

,id,funded_amount,funded_cluster,funding_goal,goal_cluster,successful_funding,success_factor,activity,sector,use,...,currency,partner_id,term_in_months,lender_count,borrower_genders,count_male,count_female,count_initiators,gender_initiators,repayment_interval
0,0,300.0,"(200, 500]",300.0,"(200, 500]",goal_achieved,0,Fruits & Vegetables,Food,"To buy seasonal, fresh fruits to sell.",...,PKR,247.0,12.0,12,female,0,1,1,female,irregular
1,1,575.0,"(500, 1000]",575.0,"(500, 1000]",goal_achieved,0,Rickshaw,Transportation,to repair and maintain the auto rickshaw used ...,...,PKR,247.0,11.0,14,"female, female",0,2,2,female,irregular
2,2,150.0,"(100, 200]",150.0,"(100, 200]",goal_achieved,0,Transportation,Transportation,To repair their old cycle-van and buy another ...,...,INR,334.0,43.0,6,female,0,1,1,female,bullet
3,3,200.0,"(100, 200]",200.0,"(100, 200]",goal_achieved,0,Embroidery,Arts,to purchase an embroidery machine and a variet...,...,PKR,247.0,11.0,8,female,0,1,1,female,irregular
4,4,400.0,"(200, 500]",400.0,"(200, 500]",goal_achieved,0,Milk Sales,Food,to purchase one buffalo.,...,PKR,245.0,14.0,16,female,0,1,1,female,monthly


In [15]:
df.success_factor = df.funded_amount/df.funding_goal

## Datentypen ändern
Wir ändern nochmal alle Datentypen in int32, um Speicherplatz einzusparen, dass neu einfügte Spalten automatisch int64 bzw. floats haben, um nicht jeden Code anzupassen.

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 666984 entries, 0 to 671204
Data columns (total 24 columns):
 #   Column              Non-Null Count   Dtype   
---  ------              --------------   -----   
 0   id                  666984 non-null  int64   
 1   funded_amount       666984 non-null  float64 
 2   funded_cluster      666984 non-null  category
 3   funding_goal        666984 non-null  float64 
 4   goal_cluster        666984 non-null  category
 5   successful_funding  666984 non-null  object  
 6   success_factor      666984 non-null  float64 
 7   activity            666984 non-null  object  
 8   sector              666984 non-null  object  
 9   use                 666973 non-null  object  
 10  use_description     666984 non-null  int64   
 11  country_code        666976 non-null  object  
 12  country             666984 non-null  object  
 13  region              614405 non-null  object  
 14  currency            666984 non-null  object  
 15  partner_id       

In [17]:
df = df.astype({"id": int, "funded_amount": int, "funding_goal": int, "use_description": int, "term_in_months": int, "lender_count": int, "count_male": int, "count_female": int, "count_initiators": int})

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 666984 entries, 0 to 671204
Data columns (total 24 columns):
 #   Column              Non-Null Count   Dtype   
---  ------              --------------   -----   
 0   id                  666984 non-null  int32   
 1   funded_amount       666984 non-null  int32   
 2   funded_cluster      666984 non-null  category
 3   funding_goal        666984 non-null  int32   
 4   goal_cluster        666984 non-null  category
 5   successful_funding  666984 non-null  object  
 6   success_factor      666984 non-null  float64 
 7   activity            666984 non-null  object  
 8   sector              666984 non-null  object  
 9   use                 666973 non-null  object  
 10  use_description     666984 non-null  int32   
 11  country_code        666976 non-null  object  
 12  country             666984 non-null  object  
 13  region              614405 non-null  object  
 14  currency            666984 non-null  object  
 15  partner_id       

# Nicht benötigte Spalten löschen

In [19]:
# Bei der Visualisierung beschränken wir uns auf bestimmte Variablen und Fragestellungen. 
# Deshalb werden alle Spalten, die nicht weiter in die Visualisierung einfließen sollen (unter anderem aus Zeitgründen)
# der Übersichtlichkeit und Performance halber gelöscht. Hierunter fällt auch die Spalte country_code, die wir zunächst
# anhand des Countries befüllt haben. Der Code zum Auffüllen der Missings in country_code aus dem Dokument 01 
# ist daher eigentlich überflüssig, da die Spalte hier gelöscht wird, wurde jedoch aus Dokumentationszwecken beibehalten.

df.drop(columns=['id', 'borrower_genders', 'use', 'currency', 'term_in_months', 'country_code', 'region'], inplace=True)

In [20]:
df.head()

,funded_amount,funded_cluster,funding_goal,goal_cluster,successful_funding,success_factor,activity,sector,use_description,country,partner_id,lender_count,count_male,count_female,count_initiators,gender_initiators,repayment_interval
0,300,"(200, 500]",300,"(200, 500]",goal_achieved,1.0,Fruits & Vegetables,Food,1,Pakistan,247.0,12,0,1,1,female,irregular
1,575,"(500, 1000]",575,"(500, 1000]",goal_achieved,1.0,Rickshaw,Transportation,1,Pakistan,247.0,14,0,2,2,female,irregular
2,150,"(100, 200]",150,"(100, 200]",goal_achieved,1.0,Transportation,Transportation,1,India,334.0,6,0,1,1,female,bullet
3,200,"(100, 200]",200,"(100, 200]",goal_achieved,1.0,Embroidery,Arts,1,Pakistan,247.0,8,0,1,1,female,irregular
4,400,"(200, 500]",400,"(200, 500]",goal_achieved,1.0,Milk Sales,Food,1,Pakistan,245.0,16,0,1,1,female,monthly


In [21]:
# exportieren als csv

df.to_csv('crowdfunding_preprocess2.csv')